# Reading in the data

In [1]:
import pandas as pd

#env_data = pd.read_csv("C:/kool/thesis/sphere_1m_env/ENV_labeled_dates_edited.csv", index_col = 0)      #loading in the data
env_data = pd.read_csv("C:/kool/thesis/sphere_1m_env/ENV_all_edited.csv", index_col = 0)
env_data['datetime'] =  pd.to_datetime(env_data['datetime'], format="%Y-%m-%d %H:%M:%S")

video_data = pd.read_csv("C:/kool/thesis/sphere_1m_env/video_all.csv", index_col = 0)
video_data = video_data[video_data.key == "2DCen"]
video_data['datetime'] =  pd.to_datetime(video_data['datetime'], format="%Y-%m-%dT%H:%M:%S.%fZ")

device_locations_eng = {
    'fd00::212:4b00:0:80' : 'lounge',
    'fd00::212:4b00:0:81' : 'study',
    'fd00::212:4b00:0:82' : 'kitchen',
    'fd00::212:4b00:0:83' : 'hall',
    'fd00::212:4b00:0:84' : 'bedroom',    
    'fd00::212:4b00:0:85' : 'bedroom2',
    'fd00::212:4b00:0:86' : 'bathroom', 
    'fd00::212:4b00:0:87' : 'landing',
    'toilet' : 'toilet',
    'stairs' : 'stairs',
    'porch' : 'porch',
    'Unknown' : 'unknown'
}

device_locations_est = {
    'fd00::212:4b00:0:80' : 'elutuba',
    'fd00::212:4b00:0:81' : 'kabinet',
    'fd00::212:4b00:0:82' : 'köök',
    'fd00::212:4b00:0:83' : 'koridor1',
    'fd00::212:4b00:0:84' : 'magamistuba',   
    'fd00::212:4b00:0:85' : 'magamistuba2',
    'fd00::212:4b00:0:86' : 'vannituba',  
    'fd00::212:4b00:0:87' : 'koridor2',
    'toilet' : 'WC',
    'stairs' : 'trepp',
    'porch' : 'esik',
    'Unknown' : 'teadmata'
}

C:\Users\Unknown user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Unknown user\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
video_data.head()

,datetime,device_id,key,value
1,2017-04-25 19:20:46.354,b8aeed75e024,2DCen,"[-2147483648, -2147483648]"
7,2017-04-25 19:20:46.574,b8aeed75e024,2DCen,"[257, 114]"
13,2017-04-25 20:25:39.989,b8aeed75e024,2DCen,"[-2147483648, -2147483648]"
19,2017-04-25 20:29:55.334,b8aeed75e024,2DCen,"[-2147483648, -2147483648]"
25,2017-04-25 20:29:55.544,b8aeed75e024,2DCen,"[444, 357]"


# Functions for location predictions

## The main function to get results: getLocationPredictions(env_data, start, end, wrong_signal_count = 1)  
env_data - It's the environment data that is a used for making the predictions.  
start - It's the datetime what is the starting datetime for giving predictions (suitable format example: "2017-04-30 9:00:00")  
end - It's the datetime what is the ending datetime for giving predictions (suitable format example: "2017-04-30 9:00:00")  
wrong_signal_count - It shows maximally, how many motion detections by one device in a row is considered as a possible false signal. If it is possible to remove the possible false signal without making the person to teleport, then the model will remove it.  
landing_dur_for_toilet_try - It indicates the minimum length that the person has to be detected in landing in order to try placing the person into toilet.  
min_toilet_visit - It indicates how long the signal has to be missing in the landing in a row so that we can assume that the person really is in toilet.  
toilet_delay - It indicates how many seconods will be added to the last signal in the landing, when we assume that the person has gone to the toilet.  
hall_dur_for_porch_try - It indicates the minimum length that the person has to be detected in the hall in order to try placing the person into the toilet.
min_porch_visit - It indicates how long the signal has to be missing in the hall in a row so that we can assume that the person really is in the porch.  
porch_delay - It indicates how many seconods will be added to the last signal in the hall, when we assume that the person has gone to the porch.   
first_to_ground_stairs_delay - It indicates how many seconds we have to increase to the point where the landing signal changed into hall signal in order to get the start time for stairs(negative number for decreasing).  
first_to_ground_hall_delay - It indicates how many seconds we have to increase to the point where the landing signal changed into hall signal in order to get the real start time for hall.  
ground_to_first_stairs_delay - It indicates how many seconds we have to increase to the point where the hall signal changed into landing signal in order to get the start time for stairs(negative number for decreasing).  
ground_to_first_landing_delay - It indicates how many seconds we have to increase to the point where the hall signal changed into landing signal in order to get the real start time for landing.  

How location predictions work:
1. I'm taking env data
    * selecting where pir_trigs == 1
    * finding 20 last triggered instances before wished datetime
    * finding all triggered instances between wanted datetime
    * finding first 10 triggered instances after wished datetime
    * Joining all the data together
2. Creating occurances dataframe which shows how many times one device detected motion in a row (it gives also device id and the datetime of first motion)
3. I start looking occurances dataframe row by row and return my first location predictions.
    * for each row I check if the occurance count is smaller or equals with wrong_signal_count(a parameter in main function).
        * If the count is smaller or equal then I remove it if it was possible to get from previous room into next room. (only exception is a moment, when on room change two sensors detected motion at the same time and then I prefer the new room)
        * Otherwise I fill add the room into our result.
4. Now I start adding toilet, stairs and 
    * If the person is in the landing long enough (there's a parameter for determing it) and the signal is missing during that time long enough (also a parameter for that) then we will add toilet.
    * If the person moves from landing to hall or from hall to landing then we will add stairs between them.
    * If the person goes to hall and disapears from there for some time, then I will add porch. (It also might happen when the person is in lounge according to pir sensors)
5. Now I change the border datetimes. Previously I took more data, than I was asked to so I can make better predictions on exact starting and ending time. Now I remove the predictions from the borders and return exactly the information that was between given borders.
6. I will check if all the room changes are possible I print some messages if they aren't so I would know that something is wrong.

In [2]:
from datetime import datetime, timedelta


def filterDatetime(data, start, end):
    if not isinstance(start, datetime):                    # Checking if converting is needed or not
        start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
    if not isinstance(end, datetime):
        end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    return data[(data.datetime >= start) & (data.datetime <= end)]


def getDataBefore(data, time):
    if not isinstance(time, datetime):                    # Checking if converting is needed or not
        time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    result = data[data.datetime < time]
    return result.sort_values(["datetime"], ascending=False)


def getDataAfter(data, time):
    if not isinstance(time, datetime):                    # Checking if converting is needed or not
        time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    result = data[data.datetime > time]
    return result.sort_values(["datetime"], ascending=True)


def prepareEnvData(data, start, end):                        # selects all the important data for making room predictions
    data = data[(data.sensor == "PIR_TRIGS") & (data.value == 1)]
    data_before = getDataBefore(data, start).head(30)         # 30 instances brfore start
    data_between = filterDatetime(data, start, end)
    data_after = getDataAfter(data, end).head(20)             # 20 instances after end
    merged_result = pd.concat([data_before, data_between, data_after])
    return merged_result.sort_values(["datetime"], ascending=True)    
    
    
def createOccurenceDf(data):                              # creates dataframe that shows starting time for one room
    datetimes = []                                        # and consecuent signal count
    devices = []
    occurances = []
    
    counter = 0
    last_device = None
    first_datetime = None
    for index, row in data.iterrows():
        device = row["device_id"]
        if device == last_device:
            counter += 1
        else:
            if last_device != None:                # new device, putting last device information into lists
                datetimes.append(first_datetime)
                devices.append(last_device)
                occurances.append(counter)
            dt = row["datetime"]                   # resetting device primitive information
            first_datetime = dt
            last_device = device
            counter = 1
            
    if last_device != None:                # after the loop ends I put the last device into lists
        datetimes.append(first_datetime)
        devices.append(last_device)
        occurances.append(counter)
    
    result = pd.DataFrame({'device_id' : devices,                # creating dataframe from lists
                          'datetime' : datetimes,
                          'occurance_count' : occurances})
    return result


def getLocResults(occurances, env_data, wrong_signal_count):                  # It creates the initial predictions while
    device_and_time = []                                                      # removing short room signals if possible
    last_device = None
    should_be_added = True
    for index, row in occurances.iterrows():
        dt = row['datetime']
        device = row['device_id']
        occ_count = row['occurance_count']
        if occ_count <= wrong_signal_count:                          # too few signals, maybe the room shouldn't be added                 
            should_be_added = shouldBeAdded(occurances, env_data, index, wrong_signal_count)
            if should_be_added and (len(device_and_time) == 0 or device != device_and_time[-1][0] ):
                device_and_time.append([device, dt])
        elif len(device_and_time) == 0 or device != device_and_time[-1][0]:    # avoids predicting same room twice in a row 
            device_and_time.append([device, dt])                               # and always adds the first device
    return device_and_time


def shouldBeAdded(occurances, env_data, index, wrong_signal_count):          # Helps the previous method to determine wether we 
    row = occurances.iloc[index]                                             # should add device or not
    prev_row = occurances.iloc[index - 1]
    next_row = occurances.iloc[index + 1]
    if (prev_row["device_id"] == next_row["device_id"]):        
        # next thing happens when two rooms detect motion at the same time (I consider it as going into a new room)
        if (next_row["datetime"] == row["datetime"] and next_row["occurance_count"] <= wrong_signal_count and 
                            row["device_id"] in ['fd00::212:4b00:0:87', 'fd00::212:4b00:0:83']):
            return True
        return False
    return not possibleLocChange(prev_row["device_id"], next_row["device_id"])   # watching if the person will be teleportin if
                                                                                 # I don't add the room
    
def possibleLocChange(last, current):
    if last == None or last == current:
        return True
    # Checking possible movements on the first floor
    if last == 'fd00::212:4b00:0:87' and current in ['fd00::212:4b00:0:84', 'fd00::212:4b00:0:85', 'fd00::212:4b00:0:86']:
        return True
    if current == 'fd00::212:4b00:0:87' and last in ['fd00::212:4b00:0:84', 'fd00::212:4b00:0:85', 'fd00::212:4b00:0:86']:
        return True
    # Checking possible movements on the ground floor
    if last == 'fd00::212:4b00:0:83' and current in ['fd00::212:4b00:0:80', 'fd00::212:4b00:0:81', 'fd00::212:4b00:0:82']:
        return True
    if current == 'fd00::212:4b00:0:83' and last in ['fd00::212:4b00:0:80', 'fd00::212:4b00:0:81', 'fd00::212:4b00:0:82']:
        return True
    # Checking possible movements betwwen the floors
    if current == 'fd00::212:4b00:0:83' and last == 'fd00::212:4b00:0:87' or current == 'fd00::212:4b00:0:87' and last == 'fd00::212:4b00:0:83':
        return True
    return False


def addNoPirRooms(loc_results, env_data, video_data, landing_dur_for_toilet_try, min_toilet_visit, toilet_delay,
                 toilet_end_delay, min_bedroom2_visit, hall_dur_for_porch_try, min_porch_visit, porch_delay, porch_end_delay, 
                 min_lounge_time, first_to_ground_stairs_delay, first_to_ground_hall_delay, ground_to_first_stairs_delay, 
                  ground_to_first_landing_delay, bath_to_ground_stairs_delay, bath_to_ground_hall_delay):
    new_results = []
    last_device = None
    last_dt = None
    for result in loc_results:
        device = result[0]
        dt = result[1]
        if last_dt != None:
            time_diff = (dt - last_dt).total_seconds()
            # checking If I should add toilet
            if last_device == "fd00::212:4b00:0:87" and time_diff > landing_dur_for_toilet_try:
                toilet_results = checkIfToiletShouldBeAdded(env_data, last_dt, dt, min_toilet_visit, toilet_delay, 
                                                            toilet_end_delay, min_bedroom2_visit)
                if toilet_results != None:
                    for item in toilet_results:
                        new_results.append(item)

            # checkin if I should add porch
            if ((last_device == "fd00::212:4b00:0:83" and time_diff > hall_dur_for_porch_try) 
                            or last_device == "fd00::212:4b00:0:80"):
                porch_results = checkIfPorchShouldBeAdded(env_data, video_data, last_device, last_dt, dt, min_porch_visit, porch_delay,
                                                         porch_end_delay, min_lounge_time)
                if porch_results != None:
                    for item in porch_results:
                        new_results.append(item)

        # I start checking if I should add stairs
        if last_device == "fd00::212:4b00:0:87" and device == "fd00::212:4b00:0:83":  # from firsr floor to ground floor
            new_results.append(['stairs', dt + timedelta(0, first_to_ground_stairs_delay)])
            new_results.append([device, dt + timedelta(0, first_to_ground_hall_delay)])
        elif device == "fd00::212:4b00:0:87" and last_device == "fd00::212:4b00:0:83":      # from ground floor to first floor
            new_results.append(['stairs', dt + timedelta(0, ground_to_first_stairs_delay)])
            new_results.append([device, dt + timedelta(0, ground_to_first_landing_delay)])
        elif last_device == "fd00::212:4b00:0:86" and device == "fd00::212:4b00:0:83":       # from bathroom to ground floor
            new_results.append(['stairs', dt + timedelta(0, bath_to_ground_stairs_delay)])
            new_results.append([device, dt + timedelta(0, bath_to_ground_hall_delay)])
        else:
            new_results.append(result) 

        last_device = device
        last_dt = dt
    return new_results
    

def checkIfToiletShouldBeAdded(env_data, start_dt, end_dt, min_toilet_visit, toilet_delay, toilet_end_delay, min_bedroom2_visit):
    data = env_data[(env_data.datetime >= start_dt) & (env_data.datetime < end_dt)]
    toilet_start = None
    toilet_end = None
    last_dt = None
    for index, row in data.iterrows():
        dt = row["datetime"]
        if last_dt != None:
            time_diff = (dt - last_dt).total_seconds()
            if time_diff > min_bedroom2_visit:                        # he went to second bedroom (it might be never used)
                return [['fd00::212:4b00:0:86', last_dt], ['fd00::212:4b00:0:87', dt]]
            if time_diff > min_toilet_visit:                           #Let's say that the person is visiting toile
                if toilet_start == None:
                    toilet_start = last_dt + timedelta(0, toilet_delay)
                toilet_end = dt
        last_dt = dt
    if toilet_start != None:
        return [['toilet', toilet_start], ['fd00::212:4b00:0:87', toilet_end + timedelta(0, toilet_end_delay)]]
    return None                     # I'm returning None if the person didn't visit toilet


def checkIfPorchShouldBeAdded(env_data, video_data, last_device, start_dt, end_dt, min_porch_visit, porch_delay, porch_end_delay,
                             min_lounge_time):
    hall_video = video_data[video_data.device_id == "b8aeed75e024"]
    hall_video = hall_video[(hall_video.datetime > start_dt) & (hall_video.datetime < end_dt)]
    hall_video = hall_video.sort_values(["datetime"], ascending=True)
    if last_device == "fd00::212:4b00:0:83":    # checking if the person disaeppears from hall long enough
        porch_start = None
        porch_end = None
        last_dt = None 
        for index, row in hall_video.iterrows():
            dt = row['datetime']
            if last_dt != None:
                time_diff = (dt - last_dt).total_seconds()
                if time_diff > min_porch_visit:
                    if porch_start == None:
                        porch_start = datetime.strptime(last_dt.strftime("%Y-%m-%d %H:%M:%S"), 
                                                        "%Y-%m-%d %H:%M:%S") + timedelta(0, porch_delay)
                    porch_end = datetime.strptime(dt.strftime("%Y-%m-%d %H:%M:%S"), 
                                                  "%Y-%m-%d %H:%M:%S") + timedelta(0, porch_end_delay)
            last_dt = dt
        if porch_start != None:
            return [['porch', porch_start], ['fd00::212:4b00:0:83', porch_end]]
    elif last_device == "fd00::212:4b00:0:80":      #it's possible to get from lounge to porch
        hall_start = None
        porch_start = None
        porch_end = None
        last_dt = None
        for index, row in hall_video.iterrows():
            dt = row['datetime']
            if last_dt != None:
                time_diff = (dt - last_dt).total_seconds()
                if time_diff > min_porch_visit: #and len(filterDatetime(env_data, last_dt, dt).index) == 0:
                    if porch_start == None:
                        porch_start = datetime.strptime(last_dt.strftime("%Y-%m-%d %H:%M:%S"), 
                                                        "%Y-%m-%d %H:%M:%S") + timedelta(0, porch_delay)
                    porch_end = datetime.strptime(dt.strftime("%Y-%m-%d %H:%M:%S"), 
                                                  "%Y-%m-%d %H:%M:%S") + timedelta(0, porch_end_delay)
            if hall_start == None and dt + timedelta(0, min_lounge_time) > dt:
                hall_start = datetime.strptime(dt.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
            last_dt = dt
        if porch_end != None and len(filterDatetime(env_data, porch_start, porch_end).index) == 0:
            return [['fd00::212:4b00:0:83', hall_start], ['porch', porch_start], ['fd00::212:4b00:0:83', porch_end]]    
    return None
    
    
def changeTimings(loc_results, study_to_hall_change, kitchen_to_hall_change, lounge_to_hall_change, 
                 lounge_change, study_change, kitchen_change, bath_to_landing_change, bed_to_landing_change, 
                 bed2_to_landing_change, bedroom_change, bedroom2_change, bathroom_change):
    device_and_time = []
    last_device = None
    last_dt = datetime.strptime("1000-01-01 10:00:00", "%Y-%m-%d %H:%M:%S")
    for result in loc_results:
        device = result[0]
        dt = result[1]
        if device == last_device or last_dt > dt:
            last_device = device
            pass
        elif device == "fd00::212:4b00:0:80":
            device_and_time.append([device, dt + timedelta(0, lounge_change)])
        elif device == "fd00::212:4b00:0:81":
            device_and_time.append([device, dt + timedelta(0, study_change)])
        elif device == "fd00::212:4b00:0:82":
            device_and_time.append([device, dt + timedelta(0, kitchen_change)])
        elif device == "fd00::212:4b00:0:83" and last_device == "fd00::212:4b00:0:81":
            device_and_time.append([device, dt + timedelta(0, study_to_hall_change)])
        elif device == "fd00::212:4b00:0:83" and last_device == "fd00::212:4b00:0:82":
            device_and_time.append([device, dt + timedelta(0, kitchen_to_hall_change)])
        elif device == "fd00::212:4b00:0:83" and last_device == "fd00::212:4b00:0:80":
            device_and_time.append([device, dt + timedelta(0, lounge_to_hall_change)])
        elif device == "fd00::212:4b00:0:84":
            device_and_time.append([device, dt + timedelta(0, bedroom_change)])
        elif device == "fd00::212:4b00:0:85":
            device_and_time.append([device, dt + timedelta(0, bedroom2_change)])
        elif device == "fd00::212:4b00:0:86":
            device_and_time.append([device, dt + timedelta(0, bathroom_change)])
        elif device == "fd00::212:4b00:0:87" and last_device == "fd00::212:4b00:0:84":
            device_and_time.append([device, dt + timedelta(0, bed_to_landing_change)])
        elif device == "fd00::212:4b00:0:87" and last_device == "fd00::212:4b00:0:85":
            device_and_time.append([device, dt + timedelta(0, bed2_to_landing_change)])
        elif device == "fd00::212:4b00:0:87" and last_device == "fd00::212:4b00:0:86":
            device_and_time.append([device, dt + timedelta(0, bath_to_landing_change)])
        else:
            device_and_time.append([device, dt])
        last_device = device
        last_dt = dt
    return device_and_time
    
def trimBorders(results, start, end):
    start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
    end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    
    last = ["Unknown", None]                       # trimming unwanted info before start time
    for i in range(len(results)):
        r = results[0]
        if r[1] < start:
            last = r
            results.remove(r)
        else:
            last[1] = start
            results.insert(0, last)
            break

    for i in range(len(results)):      # trimming unwanted results from end
        r = results[-1]
        if r[1] > end:
            results.remove(r)
        else:
            break
    return results



# It's the main method
def getLocationPredictions(env_data, video_data, start, end, wrong_signal_count = 2, landing_dur_for_toilet_try = 10, 
                           min_toilet_visit = 4, toilet_delay = 1, toilet_end_delay = 1, min_bedroom2_visit = 1000,
                           hall_dur_for_porch_try = 20, min_porch_visit = 10,  porch_delay = 2, porch_end_delay = 0, 
                           min_lounge_time = 1,
                           first_to_ground_stairs_delay = -3, first_to_ground_hall_delay = 6, ground_to_first_stairs_delay = -4, 
                           ground_to_first_landing_delay = 3, bath_to_ground_stairs_delay = -1, bath_to_first_hall_delay = 8,
                           study_to_hall_change = 0, kitchen_to_hall_change = 0, lounge_to_hall_change = -1, 
                           lounge_change = 0, study_change = 0, kitchen_change = 1, 
                           bath_to_landing_change = 0, bed_to_landing_change = 1, bed2_to_landing_change = 0, 
                           bedroom_change = 0, bedroom2_change = 0, bathroom_change = 0):
    env_data = prepareEnvData(env_data, start, end)
    pir_signals_df = createOccurenceDf(env_data)
    loc_results = getLocResults(pir_signals_df, env_data, wrong_signal_count)
    loc_results = addNoPirRooms(loc_results, env_data, video_data, landing_dur_for_toilet_try, min_toilet_visit, toilet_delay, 
                                toilet_end_delay, min_bedroom2_visit,
                                hall_dur_for_porch_try, min_porch_visit, porch_delay, porch_end_delay, min_lounge_time,
                                first_to_ground_stairs_delay, first_to_ground_hall_delay, ground_to_first_stairs_delay, 
                                ground_to_first_landing_delay, bath_to_ground_stairs_delay, bath_to_first_hall_delay)
    loc_results = changeTimings(loc_results, study_to_hall_change, kitchen_to_hall_change, lounge_to_hall_change, 
                                lounge_change, study_change, kitchen_change, bath_to_landing_change, bed_to_landing_change, 
                                bed2_to_landing_change, bedroom_change, bedroom2_change, bathroom_change)
    loc_results = trimBorders(loc_results, start, end)
    return loc_results

    
def showLocationResults(results, lang="eng"):
    i = 0
    if lang == "est":
        for result in results:
            print(str(result[1]) + " : " + device_locations_est[result[0]])
    elif lang == "eng":
        last_dt = datetime.strptime("1000-01-01 10:00:00", "%Y-%m-%d %H:%M:%S")
        last_device = "aaa"
        
        for result in results:
            dt = result[1]
            device = result[0] 
            if dt >= last_dt and device != last_device:
                print(str(result[1]) + " : " + device_locations_eng[result[0]])
                i +=1
                last_dt = dt
                last_device = device
    print("Shown predictions: " + str(i))
   

def showWeekResults(results, start, end):                              # It was useful for adding example to thesis
    last_dt = datetime.strptime("1000-01-01 10:00:00", "%Y-%m-%d %H:%M:%S")
    last_device = "aaa"
    i = 0
    
    for result in results:
        dt = result[1]
        device = result[0] 
        if dt >= last_dt and device != last_device:
            i+=1
            if i > end:
                break
            if i >= start:
                if (result[1].date() == datetime.strptime("2017-04-29", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-01 " + time + " : " + device_locations_est[result[0]]  + " \\newline")
                elif (result[1].date() == datetime.strptime("2017-04-30", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-02 " + time + " : " + device_locations_est[result[0]] + " \\newline")
                elif (result[1].date() == datetime.strptime("2017-05-01", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-03 " + time + " : " + device_locations_est[result[0]]  + " \\newline")
                elif (result[1].date() == datetime.strptime("2017-05-02", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-04 " + time + " : " + device_locations_est[result[0]]  + " \\newline")
                elif (result[1].date() == datetime.strptime("2017-05-03", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-05 " + time + " : " + device_locations_est[result[0]]   + " \\newline")
                elif (result[1].date() == datetime.strptime("2017-05-04", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-06 " + time + " : " + device_locations_est[result[0]]+ " \\newline")
                elif (result[1].date() == datetime.strptime("2017-05-05", "%Y-%m-%d").date()):
                    time = dt.strftime("%Y-%m-%d %H:%M:%S").split()[1]
                    print("2050-01-07 " + time + " : " + device_locations_est[result[0]]  + " \\newline")
                else:
                    print("mittesobiv kuupäev")
                    print(result[1].date())
            last_dt = dt
            last_device = device

In [3]:
# Example for gettin location results
predict_from = "2017-04-30 00:00:00"
predict_to = "2017-04-30 10:20:00"

results = getLocationPredictions(env_data, video_data, predict_from, predict_to)
print(len(results))
showLocationResults(results)


65
2017-04-30 00:00:00 : lounge
2017-04-30 00:31:36 : hall
2017-04-30 00:31:40 : stairs
2017-04-30 00:31:47 : landing
2017-04-30 00:31:54 : bedroom
2017-04-30 00:32:24 : landing
2017-04-30 00:32:26 : stairs
2017-04-30 00:32:35 : hall
2017-04-30 00:32:40 : kitchen
2017-04-30 00:36:51 : hall
2017-04-30 00:37:03 : stairs
2017-04-30 00:37:10 : landing
2017-04-30 00:37:14 : toilet
2017-04-30 00:37:21 : landing
2017-04-30 00:37:23 : bedroom
2017-04-30 00:37:47 : landing
2017-04-30 00:37:51 : toilet
2017-04-30 00:39:36 : landing
2017-04-30 00:39:38 : bedroom
2017-04-30 00:40:09 : landing
2017-04-30 00:40:11 : toilet
2017-04-30 00:43:44 : landing
2017-04-30 00:43:57 : stairs
2017-04-30 00:44:06 : hall
2017-04-30 00:44:19 : stairs
2017-04-30 00:44:26 : landing
2017-04-30 00:44:34 : bedroom
2017-04-30 00:51:13 : landing
2017-04-30 00:51:18 : stairs
2017-04-30 00:51:27 : hall
2017-04-30 00:51:30 : lounge
2017-04-30 00:51:52 : hall
2017-04-30 00:51:56 : stairs
2017-04-30 00:52:03 : landing
2017-04

In [23]:
# Example for getting weekly location predictions
predict_from = "2017-04-29 00:00:00"
predict_to = "2017-05-05 23:59:59"

results = getLocationPredictions(env_data, video_data, predict_from, predict_to)

#showWeekResults(results, 1, 1000)
showWeekResults(results, 10, 20)

2050-01-01 09:23:55 : koridor2 \newline
2050-01-01 09:24:03 : magamistuba \newline
2050-01-01 09:25:49 : koridor2 \newline
2050-01-01 09:25:51 : trepp \newline
2050-01-01 09:26:00 : koridor1 \newline
2050-01-01 09:26:02 : elutuba \newline
2050-01-01 09:26:20 : koridor1 \newline
2050-01-01 09:26:24 : köök \newline
2050-01-01 09:26:27 : koridor1 \newline
2050-01-01 09:26:50 : köök \newline
2050-01-01 09:40:38 : koridor1 \newline



21


# Here starts the activity prediction model

In [19]:
predict_from = "2017-04-30 00:01:00"
predict_to = "2017-04-30 15:20:00"

results = getLocationPredictions(env_data, video_data, predict_from, predict_to)

activities_eng = {
    'active' : 'active',
    'unactive' : 'unactive',
    'unknown' : 'unknown',
    'went to bed' : "went to bed",
    'came out of bed' : "came out of bed",
    'opened curtains' : "curtains were opened",
    'fridge was opened' : 'fridge was opened',
    'fridge was closed' : 'fridge was closed',
    'toaster start' : 'started toaster',
    'food to microwave' : 'put food into microwave',
    'microwave_start' : 'started microwave',
    'food from microwave' : 'took food from microwave',
    'washing machine start' : 'washing machine was started'
}

activities_est = {
    'active' : 'aktiivne tegevus',
    'unactive' : 'passiivne tegevus',
    'unknown' : 'teadmata tegevus',
    'went to bed' : "läks voodisse",
    'came out of bed' : "tuli voodist välja",
    'opened curtains' : "avas kardinad"
}

In [25]:
def predictActivities(loc_results, env_data, end, lounge_activity_limit = 10, lounge_activity_break = 4,    #main method for 
                      study_activity_limit = 3, study_activity_break = 10,                                  # activity predictions
                      bedroom_activity_limit = 10, bedroom_activity_break=3, min_bed_time = 240, 
                      curtains_light_change = 20, max_night_light = 5):
    new_results = []
    for i in range(len(results)):
        result = results[i]
        if i + 1 == len(results):  # this part is jsut for determining room end time
            next_result = None
        else:
            next_result = results[i+1]
        device = result[0]
        device_start = result[1]
        if next_result == None:
            device_end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
        else:
            device_end = next_result[1]
        # Now there will come different methods for each room
        if device == "fd00::212:4b00:0:80":                #lounge
            lounge_activities = getActivityBasedOnPIR(env_data, device_start, device_end, device, lounge_activity_limit, 
                                                     lounge_activity_break)
            for act in lounge_activities:    # adding room predictions to results like that
                act.insert(1, device)
                new_results.append(act)
        elif device == "fd00::212:4b00:0:81":             #study
            study_activities = getActivityBasedOnPIR(env_data, device_start, device_end, device, study_activity_limit, 
                                                     study_activity_break)
            for act in study_activities:
                act.insert(1, device)
                new_results.append(act)
        elif device == "fd00::212:4b00:0:84":             #bedroom
            bedroom_activities = getBedroomActivities(env_data, device_start, device_end, device, bedroom_activity_limit, 
                                                     bedroom_activity_break, min_bed_time, 
                                                     curtains_light_change, max_night_light)
            for act in bedroom_activities:
                act.insert(1, device)
                new_results.append(act)
        elif device == "fd00::212:4b00:0:82":             #kitchen
            kitchen_activities = getKitchenActivities(env_data, device_start, device_end)
            if kitchen_activities == []:
                new_results.append([device_start, device, "unknown"])
            else:
                for act in kitchen_activities:
                    act.insert(1, device)
                    new_results.append(act)
        elif device == "fd00::212:4b00:0:86":            #bathroom
            new_results.append([device_start, device, "unknown"])
        elif device == "fd00::212:4b00:0:83":                           #hall
            new_results.append([device_start, device, "active"])
        elif device == "fd00::212:4b00:0:87":                           #landing
            new_results.append([device_start, device, "active"])
        elif device == "stairs":
            new_results.append([device_start, device, "active"])
        else:
            new_results.append([device_start, device, "unknown"])
    return new_results


def addToExistingResults(results, item):     # it aads activities to the right place in results and then return the results
    if results == []:
        results.append(item)
    else:
        datetime = item[0]
        activity = item[1]
        last_r_dt = None
        for i in range(len(results)):
            r = results[i]
            r_dt = r[0]
            if last_r_dt == None:
                if datetime < r_dt:
                    results.insert(0, item)
            elif last_r_dt < datetime and datetime < r_dt:
                results.insert(i, item)
            last_r_dt = r_dt
        if datetime > last_r_dt:
            results.append(item)
    return results


# method for detecting active and unactive periods   
def getActivityBasedOnPIR(env_data, start, end, pir_device, activity_limit, activity_break):   
    data = env_data[(env_data.datetime > start - timedelta(0,2)) & (env_data.datetime < end + timedelta(0,2))]
    data = data[(data.device_id == pir_device) & (data.sensor == "PIR_TRIGS")]
    activity_periods = []
    no_activity_count = 0
    activity_count = 0
    activity_start = None
    for index, row in data.iterrows():
        dt = row["datetime"]
        v = row["value"]
        if v == 1:
            no_activity_count = 0
            activity_count += 1
            if activity_start == None:
                activity_start = dt
        else:
            no_activity_count += 1
            if no_activity_count == activity_break:      #activity break shows how many 0s we will need to end active period
                if activity_count >= activity_limit:      #activity limit shows the minimum 1s count within detected active period
                    activity_periods.append([activity_start, dt - timedelta(0, activity_break), "active"])
                activity_start = None
                activity_count = 0
    if activity_start != None: 
        activity_periods.append([activity_start, end, "active"])

    final_result = []
    act_end = None
    for activity in activity_periods:       # adding final result to the results
        if act_end != None:
            final_result.append([act_end, "unactive"])
        act_start = activity[0]
        final_result.append([act_start, "active"])
        act_end = activity[1]
    
    if final_result == []:                   # so that the results would never be empty
        return [[start, "unactive"]]
    else:
        first_act_start = final_result[0][0]
        if (first_act_start - start).total_seconds() < 5:   # changing the first activity start time a little
            final_result[0][0] = start
        else:
            final_result.insert(0, [start, "unactive"])
        if (end - act_end).total_seconds() > 5:
            final_result.append([act_end, "unactive"])
        return final_result

    
def getKitchenActivities(env_data, start, end):
    data = env_data[(env_data.datetime > start - timedelta(0,2)) & (env_data.datetime < end + timedelta(0,2))]
    elec_data = data[data.sensor == "ELEC"]
    
    results = []
    fridge_data = data[data.device_id == "03727"]
    fridge_status = "closed"
    for index,row in fridge_data.iterrows():                    # checking if I should be adding fridge activities
        v = row['value']
        dt = row['datetime']
        if v > 7 and v < 20 and fridge_status == "closed":   # fridge was opened
            fridge_status = "opened"
            results.append([row['datetime'] - timedelta(0,4), "fridge was opened"])
        elif v < 7 and fridge_status == "opened":
            fridge_status = "closed"
            results.append([row['datetime'] - timedelta(0,4), "fridge was closed"])
    
    toaster_data = data[data.device_id == "03454"]
    last_value = None
    for index, row in toaster_data.iterrows():             # checking if I should be adding toaster activities
        v = row['value']
        if last_value != None:
            if v - last_value > 800: 
                to_add = [row['datetime'] - timedelta(0,4), "toaster start"]
                results = addToExistingResults(results, to_add)
        last_value = v
        
    washing_machine_data = data[data.device_id == "03182"]  # checking if I should be adding washing machine activities
    last_value = None
    for index, row in washing_machine_data.iterrows():
        v = row['value']
        if last_value != None:
            if v - last_value > 800: 
                to_add = [row['datetime'] - timedelta(0,4), "washing machine start"]
                results = addToExistingResults(results, to_add)
        last_value = v
    
    microwave_data = data[data.device_id == "02813"]        # checking if I should be adding microwave activities
    microwave_status = "off"                                # It's definitely improvable
    for index,row in microwave_data.iterrows():
        v = row['value']
        dt = row['datetime']
        if v < 100 and v > 0  and microwave_status == "off":
            microwave_status = "open"
            to_add = [row['datetime'] - timedelta(0,4), "food to microwave"]
            results = addToExistingResults(results, to_add)
        elif v > 100 and (microwave_status == "open" or microwave_status == "off"):
            microwave_status = "on"
            to_add = [row['datetime'] - timedelta(0,4), "microwave_start"]
            results = addToExistingResults(results, to_add)
        elif v < 100 and v > 0  and microwave_status == "on":
            microwave_status = "open"
        elif (microwave_status == "open" or microwave_status == "on") and v == 0:
            microwave_status = "off"
            to_add = [row['datetime'] - timedelta(0,4), "food from microwave"]
            results = addToExistingResults(results, to_add)
    return results        

    
    
def getBedroomActivities(env_data, device_start, device_end, device, bedroom_activity_limit, 
                        bedroom_activity_break, min_bed_time, 
                        curtains_light_change, max_night_light):
    pir_activities = getActivityBasedOnPIR(env_data, device_start, device_end, device, bedroom_activity_limit, 
                                                     bedroom_activity_break)

    for i in range(len(pir_activities)):         # Adding "going into bed" and "coming out of bed" activities
        act = pir_activities[i]
        if i + 1 < len(pir_activities):
            next_act_start = pir_activities[i + 1][0]
            if act[1] == "unactive" and (next_act_start - act[0]).total_seconds() > min_bed_time * 60:  
                pir_activities[i][1] = "went to bed"
                pir_activities.insert(i + 1, [next_act_start - timedelta(0,1), "came out of bed"])
                
    data = env_data[(env_data.datetime > device_start - timedelta(0,2)) & (env_data.datetime < device_end + timedelta(0,2))] #adding curtains
    data = data[(data.device_id == device) & (data.sensor == "LT")]       
    last_dt = None
    last_vale = None
    curtains_to_add = []
    for index,row in data.iterrows():                # Adding taking curtain from window activity (probably not very reliable)
        dt = row['datetime']
        v = row['value']
        if last_dt != None:
            if (last_value < max_night_light and v - last_value > curtains_light_change):
                curtains_to_add.append(dt)
        last_dt = dt
        last_value = v
        
    last_dt = None
    for i in range(len(pir_activities)):
        dt = pir_activities[i][0]
        if last_dt != None:
            for item in curtains_to_add:
                if item > last_dt and item < dt:
                    pir_activities.insert(i, [item, "opened curtains"])
                    pir_activities.insert(i + 1, [item + timedelta(0, 1), pir_activities[i-1][1]])
                    last_dt = item + timedelta(0, 1)
                else:
                    last_dt = dt
        else:
            last_dt = dt
            
        
    return(pir_activities)
    

# just a method for showing results more nicely    
def showActivityResults(results, lang="eng"):
    if lang == "est":
        for result in results:
            print(str(result[0]) + "\t" + device_locations_est[result[1]] + "\t\t" + activities_est[result[2]])
    elif lang == "eng":
        for result in results:
            print(str(result[0]) + "\t" + device_locations_eng[result[1]] + "\t\t" + activities_eng[result[2]])
       
    


In [27]:
# example for getting activity results
act_results = predictActivities(results, env_data, predict_to)  # results = room prediction results which has to be loaded in first
showActivityResults(act_results)


#data = denv_data[(env_data.datetime == "fd00::212:4b00:0:80") | (env_data.device_id == "fd00::212:4b00:0:4")]

2017-04-29 00:00:00	bedroom		went to bed
2017-04-29 08:45:46	bedroom		came out of bed
2017-04-29 08:45:47	bedroom		active
2017-04-29 08:46:09	landing		active
2017-04-29 08:46:11	toilet		unknown
2017-04-29 08:52:45	landing		active
2017-04-29 08:52:46	bedroom		active
2017-04-29 08:54:26	landing		active
2017-04-29 08:54:42	bathroom		unknown
2017-04-29 09:23:13	landing		active
2017-04-29 09:23:20	bedroom		active
2017-04-29 09:23:55	landing		active
2017-04-29 09:24:03	bedroom		unactive
2017-04-29 09:24:19	bedroom		active
2017-04-29 09:24:33	bedroom		unactive
2017-04-29 09:24:38	bedroom		active
2017-04-29 09:25:06	bedroom		unactive
2017-04-29 09:25:20	bedroom		active
2017-04-29 09:25:49	landing		active
2017-04-29 09:25:51	stairs		active
2017-04-29 09:26:00	hall		active
2017-04-29 09:26:02	lounge		active
2017-04-29 09:26:20	hall		active
2017-04-29 09:26:24	kitchen		unknown
2017-04-29 09:26:27	hall		active
2017-04-29 09:27:04	kitchen		fridge was opened
2017-04-29 09:28:06	kitchen		fridge was c